In [1]:
import MDAnalysis
import numpy as np
u = MDAnalysis.Universe("p41.pdb")

/usr/local/lib64/python3.8/site-packages/MDAnalysis/topology/PDBParser.py:330: UserWarning: Element information is absent or missing for a few atoms. Elements attributes will not be populated.
  warnings.warn("Element information is absent or missing for a few "


In [2]:
"""
Mimic the VMD command "measure minmax"
"""
def measure_minmax(atom_positions):
    xyz_array = np.transpose(atom_positions)
    min_x = np.min(xyz_array[0])
    max_x = np.max(xyz_array[0])
    min_y = np.min(xyz_array[1])
    max_y = np.max(xyz_array[1])
    min_z = np.min(xyz_array[2])
    max_z = np.max(xyz_array[2])
    return np.array([[min_x, min_y, min_z],[max_x, max_y, max_z]])


"""
Mimic the VMD command "measure center"
"""
def measure_center(atom_positions):
    xyz_array = np.transpose(atom_positions)
    center_x = np.average(xyz_array[0])
    center_y = np.average(xyz_array[1])
    center_z = np.average(xyz_array[2])
    return np.array([center_x, center_y, center_z])


def get_cell(atom_positions):
    minmax_array = measure_minmax(atom_positions)
    vec = minmax_array[1] - minmax_array[0]
    cell_basis_vector1 = np.array([vec[0], 0, 0])
    cell_basis_vector2 = np.array([0, vec[1], 0])
    cell_basis_vector3 = np.array([0, 0, vec[2]])
    return np.array([cell_basis_vector1,
                     cell_basis_vector2,
                     cell_basis_vector3])


all_atoms = u.select_atoms("all")
dist_array = all_atoms.positions
print(measure_center(dist_array))
print(get_cell(dist_array))

[ 0.031145    0.02017765 -0.13960628]
[[49.47299957  0.          0.        ]
 [ 0.         49.53600311  0.        ]
 [ 0.          0.         49.49700165]]


In [3]:
# access the current timestep
u.trajectory[0].triclinic_dimensions = get_cell(dist_array)

In [4]:
all_atoms.write("p41_gmx.gro")